In [ ]:
%matplotlib inline

### 有些没做出来的error evaluation图和network做调整tuning的参数，请钟哥帮忙调一下

https://nextjournal.com/gkoehler/pytorch-mnist evaluation的图我是看这个tutorial写的

* 第一题要做一个training dataset时候的train和test error的图，我没做出来，test_counter size不对，报错
* 第二题要fine tuning至少三个参数： kernel size, batch size, add a dropout layer + Overall, plan to evaluate 50-100 network variations (again, automate this process).
* 第三个也要一个training error的图 + 打出来modified network


* 最基础的问题就是：我没搞明白DL里面各种tensor的shape还有network里面的weights，in out channels layers这些数字
* 明天希望你能跟我重点讲一下这个部分，至少我可以独立建一个network跑dataset
* 感谢！

# 实验一

## 导入python包

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


## 数据输入pipline


### dataset

In [ ]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
training_data

In [ ]:
len(test_data)

In [ ]:
test_data[0][1]

In [ ]:
len(training_data)

In [ ]:
len(training_data[0])

In [ ]:
training_data[0][1]


In [ ]:
training_data[0][0].shape,training_data[0][1]

### dataloader


In [ ]:
batch_size = 64 # 64 samples assigned to each batch, used in forward and backward pass

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [ ]:
## N=batch size, C=channel=1=grey, Pixels: HW --> H=height, W=width
## 问题：X是test_dataloader里面的的features？y是target？
## 什么时候用enumerate（test_dataloader）?

In [ ]:

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") 
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

## 确定GPU CPU使用情况

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


## 模型




### 模型构建



In [ ]:
## 数字怎么设定的？
## conv1: in=1, out=6,ks=5
## pool:
## conv2: in=6, out=16,ks=5
## flatten???
## fc1
## fc2
## fc3



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class NetMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  
        self.pool = nn.MaxPool2d(2, 2)   ## devided by 2
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_cnn_mnist = NetMnist().to(device)
model_cnn_mnist(X).shape

### 分配损失函数和优化器


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn_mnist.parameters(), lr=1e-3)

### 构建训练和测试循环


In [ ]:
n_epochs = 3
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_dataloader.dataset) for i in range(n_epochs + 1)]

In [ ]:
len(train_dataloader.dataset)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_losses.append(loss.item())
        train_counter.append((batch*64) + ((n_epochs-1)*len(dataloader.dataset)))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_losses.append(test_loss)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 训练和测试



In [ ]:

for t in range(n_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
#     test_counter.append(i*len(train_dataloader.dataset))
    train(train_dataloader, model_cnn_mnist, loss_fn, optimizer)
    test(test_dataloader, model_cnn_mnist, loss_fn)
print("Done!")

In [ ]:
test_counter

## 模型保存和加载

In [ ]:
torch.save(model_cnn_mnist.state_dict(), "model_cnn_mnist.pth")
print("Saved PyTorch Model State to model_cnn_mnist.pth")

In [ ]:
model_cnn_mnist_load = NetMnist().to(device)
model_cnn_mnist_load.load_state_dict(torch.load("model_cnn_mnist.pth"))

## 模型预测

In [ ]:
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
]


In [ ]:
model_cnn_mnist_load.eval()
for i in range(9):
    x,y = test_data[i][0], test_data[i][1]
    with torch.no_grad():
        pred = model_cnn_mnist_load(x.unsqueeze(0))
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')
        one_img = test_data[i][0].numpy()
        plt.imshow(one_img.squeeze(), cmap='Greys')
        plt.show()

In [ ]:
## Evaluation

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
fig


# 实验二

## 导入python包

In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import torchvision
import torchvision.transforms as transforms

## 数据输入pipline


### dataset

In [16]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [18]:
len(training_data)

60000

In [19]:
len(training_data[0])

2

In [20]:
training_data[0][0].shape,training_data[0][1]

(torch.Size([1, 28, 28]), 9)

### dataloader


In [21]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [22]:

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


## 确定GPU CPU使用情况

In [23]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


## 模型




### 模型构建



In [38]:
import torch.nn as nn
import torch.nn.functional as F


class NetFMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_cnn_fmnist = NetFMnist().to(device)
model_cnn_fmnist(X).shape

torch.Size([64, 10])

### 分配损失函数和优化器


In [39]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn_fmnist.parameters(), lr=1e-3)

### 构建训练和测试循环


In [40]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [41]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 训练和测试



In [42]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_cnn_fmnist, loss_fn, optimizer)
    test(test_dataloader, model_cnn_fmnist, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308301  [    0/60000]
loss: 0.879384  [ 6400/60000]
loss: 0.574974  [12800/60000]
loss: 0.805010  [19200/60000]
loss: 0.618829  [25600/60000]
loss: 0.579969  [32000/60000]
loss: 0.615681  [38400/60000]
loss: 0.555993  [44800/60000]
loss: 0.604533  [51200/60000]
loss: 0.585751  [57600/60000]
Test Error: 
 Accuracy: 80.3%, Avg loss: 0.531690 

Done!


## 模型保存和加载

In [ ]:
torch.save(model_cnn_fmnist.state_dict(), "model_cnn_fmnist.pth")
print("Saved PyTorch Model State to model_cnn_fmnist.pth")

In [ ]:
model_cnn_fmnist_load = NetFMnist().to(device)
model_cnn_fmnist_load.load_state_dict(torch.load("model_cnn_fmnist.pth"))

## Output Class

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


In [ ]:
model_cnn_fmnist_load.eval()
for i in range(9):
    x,y = test_data[i][0], test_data[i][1]
    with torch.no_grad():
        pred = model_cnn_fmnist_load(x.unsqueeze(0))
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')
        one_img = test_data[i][0].numpy()
        plt.imshow(one_img.squeeze(), cmap='Greys')
        plt.show()

## Change Dimension #1: Kernel Size
Change from 5 to 3
Accuracy score is changed from previous results Accuracy: 78.4%, Avg loss: 0.561791 to current results 
Accuracy: 10.0%, Avg loss: 2.304142 

In [37]:
import torch.nn as nn
import torch.nn.functional as F


class KS_NetFMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


ks_model_cnn_fmnist = KS_NetFMnist().to(device)
ks_model_cnn_fmnist(X).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x400 and 256x120)

In [ ]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, ks_model_cnn_fmnist, loss_fn, optimizer)
    test(test_dataloader, ks_model_cnn_fmnist, loss_fn)
print("Done!")

## Change Dimension #2: Add a Random Dropout Layer
Hypothesis:
Increase the accuracy score by adding a dropout layer in the end 

Actual:\
Adding a dropout layer decreases the accuracy score of the network \
Network without a dropout layer: Accuracy: 80.3%, Avg loss: 0.531690 \
Network with a dropout layer: Accuracy: 78.2%, Avg loss: 0.554693 

In [35]:
import torch.nn as nn
import torch.nn.functional as F


class add_dropout_NetFMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


add_dropout_model_cnn_fmnist = add_dropout_NetFMnist().to(device)
add_dropout_model_cnn_fmnist(X).shape

torch.Size([64, 10])

In [36]:
add_dropout_model_cnn_fmnist = add_dropout_NetFMnist().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(add_dropout_model_cnn_fmnist.parameters(), lr=1e-3)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, add_dropout_model_cnn_fmnist, loss_fn, optimizer)
    test(test_dataloader, add_dropout_model_cnn_fmnist, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294021  [    0/60000]
loss: 0.951911  [ 6400/60000]
loss: 0.616853  [12800/60000]
loss: 0.814038  [19200/60000]
loss: 0.652619  [25600/60000]
loss: 0.575301  [32000/60000]
loss: 0.590710  [38400/60000]
loss: 0.667489  [44800/60000]
loss: 0.607063  [51200/60000]
loss: 0.566975  [57600/60000]
Test Error: 
 Accuracy: 78.2%, Avg loss: 0.554693 

Done!


## Change Dimension #3: Change Batch Size
Increasing the batch size from 32 to 64 will also increase the accuracy score of the network. \
Hypothesis: \
Increase in batch size allows the accuracy score to increase. \
Actual: \
When batch size is 32: Accuracy: Accuracy: 81.8%, Avg loss: 0.483238 \
When batch size is 64: Accuracy: Accuracy: 81.9%, Avg loss: 0.488062 

In [29]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [30]:

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [31]:
model_cnn_fmnist = NetFMnist().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn_fmnist.parameters(), lr=1e-3)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_cnn_fmnist, loss_fn, optimizer)
    test(test_dataloader, model_cnn_fmnist, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.290911  [    0/60000]
loss: 0.793741  [ 6400/60000]
loss: 0.488140  [12800/60000]
loss: 0.630861  [19200/60000]
loss: 0.689843  [25600/60000]
loss: 0.527451  [32000/60000]
loss: 0.556974  [38400/60000]
loss: 0.625162  [44800/60000]
loss: 0.584128  [51200/60000]
loss: 0.483744  [57600/60000]
Test Error: 
 Accuracy: 81.9%, Avg loss: 0.488062 

Done!


# 实验三

In [53]:
!unzip /content/greek_train.zip

unzip:  cannot find or open /content/greek_train.zip, /content/greek_train.zip.zip or /content/greek_train.zip.ZIP.


In [54]:
!unzip /Users/mucheng/Desktop/greek_train.zip 

Archive:  /Users/mucheng/Desktop/greek_train.zip
   creating: greek_train/
  inflating: __MACOSX/._greek_train  
   creating: greek_train/beta/
  inflating: __MACOSX/greek_train/._beta  
   creating: greek_train/gamma/
  inflating: __MACOSX/greek_train/._gamma  
   creating: greek_train/alpha/
  inflating: __MACOSX/greek_train/._alpha  
  inflating: greek_train/beta/beta_006.png  
  inflating: __MACOSX/greek_train/beta/._beta_006.png  
  inflating: greek_train/beta/beta_007.png  
  inflating: __MACOSX/greek_train/beta/._beta_007.png  
  inflating: greek_train/beta/beta_005.png  
  inflating: __MACOSX/greek_train/beta/._beta_005.png  
  inflating: greek_train/beta/beta_004.png  
  inflating: __MACOSX/greek_train/beta/._beta_004.png  
  inflating: greek_train/beta/beta_001.png  
  inflating: __MACOSX/greek_train/beta/._beta_001.png  
  inflating: greek_train/beta/beta_003.png  
  inflating: __MACOSX/greek_train/beta/._beta_003.png  
  inflating: greek_train/beta/beta_002.png  
  inflatin

## 模型




### 模型构建



In [55]:
import torch.nn as nn
import torch.nn.functional as F


class NetMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_cnn_greek = NetMnist().to(device)
model_cnn_greek(X).shape

torch.Size([64, 10])

### 加载模型权重并设置权重不可训练

In [56]:

model_cnn_greek.load_state_dict(torch.load("model_cnn_mnist.pth"))

<All keys matched successfully>

In [57]:
for param in model_cnn_greek.parameters():
    param.requires_grad = False

### 改变最后一层为3个神经元输出

In [58]:
model_cnn_greek.fc3 = nn.Linear(84, 3)
model_cnn_greek

NetMnist(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
)

In [61]:
# greek data set transform
class GreekTransform:
    def __init__(self):
        pass

    def __call__(self, x):
        x = torchvision.transforms.functional.rgb_to_grayscale( x )
        x = torchvision.transforms.functional.affine( x, 0, (0,0), 36/128, 0 )
        x = torchvision.transforms.functional.center_crop( x, (28, 28) )
        return torchvision.transforms.functional.invert( x )

training_set_path = '/content/greek_train' #'/Users/mucheng/Desktop/greek_train' #/Users/mucheng/Desktop/greek_train.zip 
# DataLoader for the Greek data set
greek_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder( training_set_path,
                    transform = torchvision.transforms.Compose( [torchvision.transforms.ToTensor(),
                                            GreekTransform(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,) )] ) ),
                    batch_size = 5,
                    shuffle = True )

### 分配损失函数和优化器


In [62]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn_greek.parameters(), lr=1e-3)

### 训练和测试



In [65]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(greek_train, model_cnn_greek, loss_fn, optimizer)
    test(greek_train, model_cnn_greek, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.177754  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.225925 

Epoch 2
-------------------------------
loss: 0.237177  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.222900 

Epoch 3
-------------------------------
loss: 0.260040  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.202898 

Epoch 4
-------------------------------
loss: 0.134477  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.224356 

Epoch 5
-------------------------------
loss: 0.208435  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.215312 

Epoch 6
-------------------------------
loss: 0.301545  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.193718 

Epoch 7
-------------------------------
loss: 0.333796  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.202849 

Epoch 8
-------------------------------
loss: 0.268860  [    0/   27]
Test Error: 
 Accuracy: 96.3%, Avg loss: 0.206327 

Epoch 9
----------------

## Plot Training Errors

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_dataloader.dataset) for i in range(n_epochs + 1)]

# 实验四

## 导入数据

In [67]:
import pandas as pd

In [68]:
pd_ori = pd.read_csv('/Users/mucheng/Desktop/heart.csv')
pd_ori.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [69]:
pd_ori.shape

(918, 12)

# 数据预处理

## 离散数据处理

In [70]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [71]:
pd_ori.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [72]:
cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value=-99))
cat_ohe_step = ('ohe', OrdinalEncoder()) #离散数据整数化处理
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = ['Sex','ChestPainType','RestingECG','ExerciseAngina'
,'ST_Slope']
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)

In [73]:
X_kdd_transformed = ct.fit_transform(pd_ori)
X_kdd_transformed.shape

(918, 5)

In [74]:
ct

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('si',
                                                  SimpleImputer(fill_value=-99,
                                                                strategy='constant')),
                                                 ('ohe', OrdinalEncoder())]),
                                 ['Sex', 'ChestPainType', 'RestingECG',
                                  'ExerciseAngina', 'ST_Slope'])])

In [75]:
ct.named_transformers_['cat'].named_steps['ohe'].categories_


[array(['F', 'M'], dtype=object),
 array(['ASY', 'ATA', 'NAP', 'TA'], dtype=object),
 array(['LVH', 'Normal', 'ST'], dtype=object),
 array(['N', 'Y'], dtype=object),
 array(['Down', 'Flat', 'Up'], dtype=object)]

In [76]:
X_kdd_transformed[:5]

array([[1., 1., 1., 0., 2.],
       [0., 2., 1., 0., 1.],
       [1., 1., 2., 0., 2.],
       [0., 0., 1., 1., 1.],
       [1., 2., 1., 0., 2.]])

In [77]:
pd_ori[cat_cols][:5]

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
0,M,ATA,Normal,N,Up
1,F,NAP,Normal,N,Flat
2,M,ATA,ST,N,Up
3,F,ASY,Normal,Y,Flat
4,M,NAP,Normal,N,Up


In [78]:
X_kdd_transformed.shape


(918, 5)

## 连续数据处理

In [79]:
pd_ori.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [80]:
pd_ori.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [81]:
num_cols = ['Age', 'RestingBP', 'Cholesterol','FastingBS','MaxHR', 'Oldpeak']

In [82]:

num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler()) #均值-标准差归一化
num_steps = [num_si_step, num_ss_step]

num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=num_transformers)

X_num_transformed = ct.fit_transform(pd_ori)
X_num_transformed.shape

(918, 6)

In [83]:
X_num_transformed[:5]

array([[-1.4331398 ,  0.41090889,  0.82507026, -0.55134134,  1.38292822,
        -0.83243239],
       [-0.47848359,  1.49175234, -0.17196105, -0.55134134,  0.75415714,
         0.10566353],
       [-1.75135854, -0.12951283,  0.7701878 , -0.55134134, -1.52513802,
        -0.83243239],
       [-0.5845565 ,  0.30282455,  0.13903954, -0.55134134, -1.13215609,
         0.57471149],
       [ 0.05188098,  0.95133062, -0.0347549 , -0.55134134, -0.5819814 ,
        -0.83243239]])

## 合并数据处理

In [84]:
transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)

X = ct.fit_transform(pd_ori)
X.shape

(918, 11)

In [85]:
dim_num = X.shape[-1]
dim_num

11

## label数据处理

In [86]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
labels_ori_list =pd_ori['HeartDisease'].values
label_list = label_encoder.fit_transform(labels_ori_list)

label_list[:10],pd_ori['HeartDisease'].values[:10]

(array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0]), array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0]))